# Hand-Eye Calibration

This notebook provides hand-eye calibration for the AprilTag minimal working example. It is intended for development use only. The resulting hand-eye transformation matrix (`hand_eye_calibration.npy`) will be hard-coded into the main notebook (src\ac_training_lab\apriltag_hardware_demo\apriltag_hardware_demo.ipynb) for system operation.

## Requirements

In [ ]:
%pip install numpy scipy opencv-python pillow matplotlib gradio-client

## Connecting to the Cobot

We first define some helper functions to display result and images. 

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import json

def display_image(image_path):
	try:
		img = Image.open(image_path).convert("RGB")
		plt.imshow(img)
		plt.title("Cobot view")
		plt.show()
	except Exception as e:
		print(f"An error occurred: {e}")

def display_result(result):
	queue_status_str = result[-1].replace('\n', ' ')
	print(f"queue status: {queue_status_str}")
	print(f"response json: {None if result[0] is None else json.loads(result[0])}")
	if len(result) == 3:
		if result[1] is None:
			return
		display_image(result[1]['value'])

Connect to the cobot to begin data collection and calibration.

In [ ]:
from gradio_client import Client
import uuid
import getpass  

USER_ID = str(uuid.uuid4())
print(f"Your user id: {USER_ID}")

hf_token = getpass.getpass("Enter your Hugging Face Token:")

client = Client(
    "AccelerationConsortium/cobot280pi-gradio-g9sv",
    hf_token=hf_token
)

client.view_api()

result = client.predict(
    user_id=USER_ID,
    api_name="/enter_queue"
)

print(result)

## Data Collection Instructions


Prepare a checkerboard pattern and attach it to a wall or stable surface.

In this section, run the code cells approximately 20 times.  
For each sample:

1. Adjust the cobot using first two cells so that the entire checkerboard is clearly visible in the image.
2. Run the third cell to save the image and corresponding cobot pose into a JSON file for hand-eye calibration.

Try to capture images from diverse positions and angles to improve calibration accuracy.



### 1. Cell to Adjust Cobot Angle

In [ ]:
# Cell to adjust by angle
result = client.predict(
	user_id=USER_ID,
	angle0 = 0,
	angle1 = 20,
	angle2 = 0,
	angle3 = 0,
	angle4 = 0,
	angle5 = 0,
	movement_speed =50,
	api_name="/control_angles"
)
display_result(result)

result = client.predict(
	user_id=USER_ID,
	api_name="/query_camera"
)
print(result)
display_result(result)

### 2. Cell to Adjust Cobot Coordinates

In [ ]:
# Cell to adjust by angle
result = client.predict(
    user_id=USER_ID,
    x = 10.0,  
    y = 0,
    z = 0,
    roll = 0,
    pitch = 0,
    yaw = 0,
    movement_speed = 50,
    api_name="/control_coords"
)

display_result(result)

result = client.predict(
	user_id=USER_ID,
	api_name="/query_camera"
)
print(result)
display_result(result)

### 3. Save Data into JSON

In [ ]:
import os
import json
import numpy as np
from scipy.spatial.transform import Rotation as R
import cv2

current_dir = os.path.dirname(__file__)
data_dir = os.path.join(current_dir, "apriltag_hardware_demo")
json_path = os.path.join(data_dir, "hand_eye_calibration_data.json")

if not os.path.exists(json_path):
    data_json = {"records": []}
else:
    with open(json_path, 'r') as f:
        data_json = json.load(f)

result_img = client.predict(
    user_id=USER_ID,
    api_name="/query_camera"
)
print(result_img)
display_result(result_img)

pil_image = result_img["image"]

image_array = np.array(pil_image)

if image_array.shape[-1] == 3:
    image_array = cv2.cvtColor(image_array, cv2.COLOR_RGB2BGR)
image_id = len(data_json["records"]) + 1
image_filename = f"{image_id}.jpg"
image_path = os.path.join(data_dir, image_filename)
cv2.imwrite(image_path, image_array)
print(f"Saved image to {image_path}")

result_pose = client.predict(
    user_id=USER_ID,
    api_name="/query_coords"
)
print(result_pose)
display_result(result_pose)

query_coords = result_pose  

x, y, z = [coord / 1000 for coord in query_coords[:3]]
rx_deg, ry_deg, rz_deg = query_coords[3:]
rx, ry, rz = np.radians([rx_deg, ry_deg, rz_deg])

rotation = R.from_euler('xyz', [rx, ry, rz]).as_matrix()

pose_matrix = np.eye(4)
pose_matrix[:3, :3] = rotation
pose_matrix[:3, 3] = [x, y, z]

record = {
    "image_path": image_filename,
    "pose_matrix": pose_matrix.tolist()  
}
data_json["records"].append(record)

with open(json_path, 'w') as f:
    json.dump(data_json, f, indent=2)
print(f"Record added to {json_path}")


## Main function for hand-eye calibration
Next, we define helper functions for calibration, and specify the checkerboard parameters:  
- `XX`: number of inner corners along the width  
- `YY`: number of inner corners along the height  
- `L`: size of each square (in meters)

The function will perform both camera calibration and hand-eye calibration, and save the camera intrinsics into `camera_params.npy` for later use.

In [ ]:
XX = 9  
YY = 12
L = 0.025  # need to check this !!!

In [ ]:
import os
import json
import numpy as np
import cv2
from scipy.spatial.transform import Rotation as R

current_dir = os.path.dirname(__file__)  
data_dir = os.path.join(current_dir, "apriltag_hardware_demo")

json_path = os.path.join(data_dir, "hand_eye_calibration_data.json")
camera_params_path = os.path.join(data_dir, "camera_params.npy")

def func():
    with open(json_path, 'r') as f:
        data_json = json.load(f)

    objp = np.zeros((XX * YY, 3), np.float32)
    objp[:, :2] = np.mgrid[0:XX, 0:YY].T.reshape(-1, 2)
    objp *= L

    obj_points = []  
    img_points = [] 

    R_tool = []
    t_tool = []

    criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, 30, 0.001)

    for record in data_json["records"]:
        image_path = os.path.join(data_dir, record["image_path"])

        if not os.path.exists(image_path):
            print(f"Image {image_path} not found. Skipping.")
            continue

        img = cv2.imread(image_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        size = gray.shape[::-1]

        ret, corners = cv2.findChessboardCorners(gray, (XX, YY), None)

        if ret:
            obj_points.append(objp)
            corners2 = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), criteria)
            img_points.append(corners2)

            pose_matrix = np.array(record["pose_matrix"])
            R_tool.append(pose_matrix[:3, :3])
            t_tool.append(pose_matrix[:3, 3])
        else:
            print(f"Chessboard not detected in {image_path}")

    ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
        obj_points,
        img_points,
        size,
        cameraMatrix=None,
        distCoeffs=None
    )

    camera_params = {
        "camera_matrix": K,
        "dist_coeff": dist
    }
    np.save(camera_params_path, camera_params)
    print(f"Saved camera parameters to {camera_params_path}")

    R_ce, t_ce = cv2.calibrateHandEye(R_tool, t_tool, rvecs, tvecs, cv2.CALIB_HAND_EYE_TSAI)

    return R_ce, t_ce


Finally, run `func()` to compute the hand-eye calibration and save the resulting matrices.

In [ ]:
import numpy as np
import os
R_ce, t_ce = func()

hand_eye_params = {
    "rotation_matrix": R_ce,
    "translation_vector": t_ce
}

hand_eye_path = os.path.join(data_dir, "hand_eye_calibration.npy")
np.save(hand_eye_path, hand_eye_params)
print(f"Saved hand-eye calibration to {hand_eye_path}")


## JSON Data Management

### Clear JSON File

In [ ]:
import os
import json

# Define paths
current_dir = os.path.dirname(__file__)
data_dir = os.path.join(current_dir, "apriltag_hardware_demo")
json_path = os.path.join(data_dir, "hand_eye_calibration_data.json")

# Create new empty json structure
data_json = {"records": []}

# Save empty file
with open(json_path, 'w') as f:
    json.dump(data_json, f, indent=2)

print(f"JSON file at {json_path} has been cleared.")

### Delete Previous Record

In [ ]:
import os
import json

# Load json
with open(json_path, 'r') as f:
    data_json = json.load(f)

if len(data_json["records"]) > 0:
    data_json["records"].pop()
    with open(json_path, 'w') as f:
        json.dump(data_json, f, indent=2)
    print("Last record has been removed.")
else:
    print("No records to remove.")